In [1]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define function:
    # file: the .asc file obtained straight from the dilatometer containing the temperature and the change in length
        # Make sure the .asc file is saved with ONLY a temperature column and a length change column (in that order)
    # sample: the name of the sample, useful for differentiating between graphs
        # Example: SA-540, TA = 820, Q = 1C/s
    # L0: intial length of the sample (generally L0 = 0.01 m)
    # offset: the strain equal to the formation of 1% martensite in 100% of austenite
        # This can be calculated using another notebook called 'Offset_Calculator.ipynb'
        # This can be run from this notebook
            # Just enter %run file_location
            # Example: %run 'Offset_Calculator.ipynb' (if the notebook is in the same directory as this notebook)
    # ranges: a nested list containing the temperature ranges to be used to find the linear gradients
        # Example: ranges = [[500,550],[550,600],[600,650]] <- (make sure the lists are in their own list [[],[],[]])
        
def Ts_Calculator(file,sample,L0,offset,ranges):
    
    # Read .asc file and calculate strain
    data = pd.read_csv('test.asc', delimiter = r"\s+", header = None, skiprows=4, usecols = range(1,3), error_bad_lines=False, engine='python')
    data.columns = ['T', 'x']
    data['e'] = (data['x']*10**(-6))/L0
    
    # Isolate the cooling curve (Note-to-self: this may need altering for complex treatments)
    dataset = data.tail(n = int(len(data)/2))
    Temp = np.array(dataset['T'])
    strain = np.array(dataset['e'])
    
    # Plot dilation curve 
    data.plot(x = 'T', y = 'e', figsize = (16,6), color = 'seagreen', label = 'Dilation Curve')
    plt.title(sample+' Dilation Analysis', fontsize = 16)
    plt.xlabel('Temperature (\N{DEGREE SIGN}C)', fontsize = 12)
    plt.ylabel('Strain (m/m)', fontsize = 12)
    plt.show()
    
    # Calculate gradient/s and isolate intercept with offsetted line
    intercepts = []
    m = 0
    fig, axs = plt.subplots(1, len(ranges), figsize = (16,5))
    for x in ranges: 
        i = []
        y = []
        y_off = []
        for n in range(len(dataset)):
            if round(dataset.iloc[n].loc['T'],0) == x[0]:
                e0 = dataset.iloc[n].loc['e']
            if round(dataset.iloc[n].loc['T'],0) == x[1]:
                e1 = dataset.iloc[n].loc['e']

        g = (e1-e0)/(x[1]-x[0])
        c = e0-(x[0]*g)+offset

        for T in Temp:
            y_off.append(T*g+c)
            y.append(T*g+c-offset)
          
        idx = np.argwhere(np.gradient(np.sign(np.array(y_off) - np.array(strain)))).flatten()   
        for n in range(len(idx)):
            if Temp[idx[n]] < x[0] and Temp[idx[n]] > (x[0]-250):
                i.append(round(Temp[idx[n]],1))
        try:      
            intercept = i[0]
            intercepts.append(intercept)
        except IndexError:
            return print('No intercept found for range: [',x[0],',',x[1],']')
        
        # Plot graphs showing interception points
        if len(ranges) > 1:
            axs[m].plot(dataset['T'], dataset['e'], label = 'Dilation Curve')
            axs[m].set_xlim(x[1]-300,x[1])
            axs[m].set_ylim(intercept*g+c-0.0005,x[1]*g+c+0.0005)
            axs[m].plot(Temp, y, linestyle = '-', color = 'k', linewidth = 1)
            axs[m].plot(Temp, y_off, linestyle = '--', color = 'k', linewidth = 1, label = 'Offset Fit')
            axs[m].scatter(intercept,intercept*g+c, marker = 'o', color = 'r')
            axs[m].text(intercept+25, intercept*g+c, str(intercept), fontsize = 12, bbox={'facecolor': 'white'}, color = 'red')

            axs[m].set_title('Slope Range: '+str(x[0])+' - '+str(x[1])+'\N{DEGREE SIGN}C', fontsize = 14)
            axs[m].set_xlabel('Temperature (\N{DEGREE SIGN}C)', fontsize = 12)
            if m == 0:
                axs[m].set_ylabel('Strain (m/m)', fontsize = 12)
            else:
                pass
            m = m + 1
        else:
            plt.plot(dataset['T'], dataset['e'], label = 'Dilation Curve')
            plt.xlim(x[1]-300,x[1])
            plt.ylim(intercept*g+c-0.0005,x[1]*g+c+0.0005)
            plt.plot(Temp, y, linestyle = '-', color = 'k', linewidth = 1)
            plt.plot(Temp, y_off, linestyle = '--', color = 'k', linewidth = 1, label = 'Offset Fit')
            plt.scatter(intercept,intercept*g+c, marker = 'o', color = 'r')
            plt.text(intercept+25, intercept*g+c, str(intercept), fontsize = 12, bbox={'facecolor': 'white'}, color = 'red')

            plt.title('Slope Range: '+str(x[0])+' - '+str(x[1])+'\N{DEGREE SIGN}C', fontsize = 14)
            plt.xlabel('Temperature (\N{DEGREE SIGN}C)', fontsize = 12)
            plt.ylabel('Strain (m/m)', fontsize = 12)
    
    print('Start Temperature = ', str(round(np.mean(intercepts),1)), u'\xb1',str(round(np.std(intercepts),2)), '\N{DEGREE SIGN}C')
    plt.show()
    return